In [33]:
# important packages
import json
import requests
import pandas as pd
import numpy as np
import time

# Random Stuff to Test How Pulling From Asset API Works

In [2]:
# url
api_endpoint = 'https://wax.api.atomicassets.io/atomicassets' 
# api key
api_request = '/v1/assets?collection_name=mlb.topps&limit=100' 

url = api_endpoint + api_request

In [3]:
first_pull = requests.get(url).json()

In [4]:
first_pull['data'][19]

{'contract': 'atomicassets',
 'asset_id': '1099555324394',
 'owner': 'ilwvs.wam',
 'is_transferable': True,
 'is_burnable': True,
 'collection': {'collection_name': 'mlb.topps',
  'name': '2021 Topps Series 1 Baseball',
  'img': 'QmStHeaGaVMQDncrs6KVg5Jrc9Q8chNj8u6QvCN96js2Qb',
  'author': 'mlb.topps',
  'allow_notify': True,
  'authorized_accounts': ['mlb.topps',
   '4je.m.wam',
   'config.nft',
   'unbox.nft',
   'sales.nft',
   'craft.nft',
   'premint.nft',
   'dk2ausetup11'],
  'notify_accounts': [],
  'market_fee': 0.06,
  'created_at_block': '113636159',
  'created_at_time': '1618257661500'},
 'schema': {'schema_name': 'promo',
  'format': [{'name': 'name', 'type': 'string'},
   {'name': 'rarity', 'type': 'string'},
   {'name': 'img', 'type': 'image'},
   {'name': 'variant', 'type': 'string'},
   {'name': 'artist', 'type': 'string'},
   {'name': 'legal', 'type': 'string'}],
  'created_at_block': '114322296',
  'created_at_time': '1618601222500'},
 'template': {'template_id': '10

In [5]:
# url
api_endpoint = 'https://wax.api.atomicassets.io/atomicassets' 
# api key
api_request = '/v1/assets?collection_name=mlb.topps&page=1500000&limit=1' 

url = api_endpoint + api_request

In [6]:
test_pull2 = requests.get(url).json()['data']
test_pull2[0]

{'contract': 'atomicassets',
 'asset_id': '1099525276108',
 'owner': '4.mrq.wam',
 'is_transferable': True,
 'is_burnable': True,
 'collection': {'collection_name': 'mlb.topps',
  'name': '2021 Topps Series 1 Baseball',
  'img': 'QmStHeaGaVMQDncrs6KVg5Jrc9Q8chNj8u6QvCN96js2Qb',
  'author': 'mlb.topps',
  'allow_notify': True,
  'authorized_accounts': ['mlb.topps',
   '4je.m.wam',
   'config.nft',
   'unbox.nft',
   'sales.nft',
   'craft.nft',
   'premint.nft',
   'dk2ausetup11'],
  'notify_accounts': [],
  'market_fee': 0.06,
  'created_at_block': '113636159',
  'created_at_time': '1618257661500'},
 'schema': {'schema_name': 'packs',
  'format': [{'name': 'name', 'type': 'string'},
   {'name': 'img', 'type': 'image'},
   {'name': 'video', 'type': 'image'},
   {'name': 'series', 'type': 'string'},
   {'name': 'contains', 'type': 'string'},
   {'name': 'unpack_url', 'type': 'string'},
   {'name': 'description', 'type': 'string'},
   {'name': 'legal', 'type': 'string'}],
  'created_at_bl

# Python Script

In [35]:
# make an empty pandas df
assets_database = pd.DataFrame()
sales_database = pd.DataFrame()

# each page contains one NFT asset - each request will obtain one page
page_counter = 1

# api endpoint for ASSETS
asset_endpoint = 'https://wax.api.atomicassets.io/atomicassets'

# api endpoint for SALES
sales_endpoint = 'https://wax.api.atomicassets.io/atomicmarket'



# PERFORM UNIT TESTING 
while True:
    # api key for ASSETS 
    asset_key = '/v1/assets?collection_name=mlb.topps&page={page_num}&limit=1000'.format(page_num = page_counter)
    
    # form the asset api to pass into the request
    asset_api = asset_endpoint + asset_key
    
    # pull an asset from the API  
    asset_pull = requests.get(asset_api).json()['data']
    
    # exit the loop if there are no more assets to pull
    if len(asset_pull) < 1000:
        continue
        
    # iterate through the array of assets - add each asset to a pandas csv
    
    assets_database = assets_database.append(pd.DataFrame.from_dict(asset_pull))
    
    # itearte through each of the 1000 assets in the pull and get their sales via asset id 
    for j in range(len(asset_pull)):
        # obtain the assets from the api pull request
    
        asset = asset_pull[j]
        
        # obtain the id of the current asset 
        asset_id = asset['asset_id']
        
        sales_page=1
        while True:
            # api key for SALES of a given asset
            sales_key = '/v1/sales?collection_whitelist=mlb.topps&page={page_num}&asset_id={ID}&limit=100'.format(ID= asset_id, page_num=sales_page) # see cells below 
    
            # form the sales api to pass into the request
            sales_api = sales_endpoint + sales_key
            # pull the sales of that ASSET from the API
            sales_pull = requests.get(sales_api).json()['data']

            # ADD TO PANDAS DATABASE
            sales_database=sales_database.append(pd.DataFrame.from_dict(sales_pull))
            if len(sales_pull) < 100: 
                #adding sleep function because most assets have no transactions
                time.sleep(0.45)
                break
            sales_page +=1
    
    # increment the page counter to pull a different asset on the next iteration
    page_counter += 1
        
assets_database

,contract,asset_id,owner,is_transferable,is_burnable,collection,schema,template,mutable_data,immutable_data,...,burned_at_block,burned_at_time,updated_at_block,updated_at_time,transferred_at_block,transferred_at_time,minted_at_block,minted_at_time,data,name
0,atomicassets,1099556772026,zfgau.wam,True,True,"{'collection_name': 'mlb.topps', 'name': '2021...","{'schema_name': 'promo', 'format': [{'name': '...","{'template_id': '104049', 'max_supply': '0', '...",{},{},...,None,None,131965617,1627426606500,131965617,1627426606500,131959545,1627423570500,{'img': 'QmfV5ARCxinvHWCuDEUTPUox2btuM8SZgbFRf...,Wooden Topps Coin
1,atomicassets,1099556771920,.nlbe.wam,True,True,"{'collection_name': 'mlb.topps', 'name': '2021...","{'schema_name': 'promo', 'format': [{'name': '...","{'template_id': '104049', 'max_supply': '0', '...",{},{},...,None,None,131974020,1627430809000,131974020,1627430809000,131959440,1627423518000,{'img': 'QmfV5ARCxinvHWCuDEUTPUox2btuM8SZgbFRf...,Wooden Topps Coin
2,atomicassets,1099556771754,.nlbe.wam,True,True,"{'collection_name': 'mlb.topps', 'name': '2021...","{'schema_name': 'promo', 'format': [{'name': '...","{'template_id': '104049', 'max_supply': '0', '...",{},{},...,None,None,131973998,1627430798000,131973998,1627430798000,131959284,1627423440000,{'img': 'QmfV5ARCxinvHWCuDEUTPUox2btuM8SZgbFRf...,Wooden Topps Coin
3,atomicassets,1099556771718,.nlbe.wam,True,True,"{'collection_name': 'mlb.topps', 'name': '2021...","{'schema_name': 'promo', 'format': [{'name': '...","{'template_id': '104049', 'max_supply': '0', '...",{},{},...,None,None,131991442,1627439520000,131991442,1627439520000,131959221,1627423408500,{'img': 'QmfV5ARCxinvHWCuDEUTPUox2btuM8SZgbFRf...,Wooden Topps Coin
4,atomicassets,1099556770305,gj2e2.wam,True,True,"{'collection_name': 'mlb.topps', 'name': '2021...","{'schema_name': 'promo', 'format': [{'name': '...","{'template_id': '104049', 'max_supply': '0', '...",{},{},...,None,None,131956658,1627422127000,131956658,1627422127000,131956658,1627422127000,{'img': 'QmfV5ARCxinvHWCuDEUTPUox2btuM8SZgbFRf...,Wooden Topps Coin
5,atomicassets,1099556770048,.nlbe.wam,True,True,"{'collection_name': 'mlb.topps', 'name': '2021...","{'schema_name': 'promo', 'format': [{'name': '...","{'template_id': '104049', 'max_supply': '0', '...",{},{},...,None,None,131991456,1627439527000,131991456,1627439527000,131955740,1627421668000,{'img': 'QmfV5ARCxinvHWCuDEUTPUox2btuM8SZgbFRf...,Wooden Topps Coin
6,atomicassets,1099556770045,.nlbe.wam,True,True,"{'collection_name': 'mlb.topps', 'name': '2021...","{'schema_name': 'promo', 'format': [{'name': '...","{'template_id': '104049', 'max_supply': '0', '...",{},{},...,None,None,131991470,1627439534000,131991470,1627439534000,131955719,1627421657500,{'img': 'QmfV5ARCxinvHWCuDEUTPUox2btuM8SZgbFRf...,Wooden Topps Coin
7,atomicassets,1099556770043,.nlbe.wam,True,True,"{'collection_name': 'mlb.topps', 'name': '2021...","{'schema_name': 'promo', 'format': [{'name': '...","{'template_id': '104049', 'max_supply': '0', '...",{},{},...,None,None,131991485,1627439541500,131991485,1627439541500,131955698,1627421647000,{'img': 'QmfV5ARCxinvHWCuDEUTPUox2btuM8SZgbFRf...,Wooden Topps Coin
8,atomicassets,1099556770036,prjfm.wam,True,True,"{'collection_name': 'mlb.topps', 'name': '2021...","{'schema_name': 'promo', 'format': [{'name': '...","{'template_id': '104049', 'max_supply': '0', '...",{},{},...,None,None,131955666,1627421631000,131955666,1627421631000,131955666,1627421631000,{'img': 'QmfV5ARCxinvHWCuDEUTPUox2btuM8SZgbFRf...,Wooden Topps Coin
9,atomicassets,1099556619782,4e.rc.wam,True,True,"{'collection_name': 'mlb.topps', 'name': '2021...","{'schema_name': 'promo', 'format': [{'name': '...","{'template_id': '104049', 'max_supply': '0', '...",{},{},...,None,None,131902251,1627394917000,131902251,1627394917000,131902251,1627394917000,{'img': 'QmfV5ARCxinvHWCuDEUTPUox2btuM8SZgbFRf...,Wooden Topps Coin


In [37]:
#Testing uniqueness of data

#assets_database.iloc[1000]
#len(pd.unique(assets_database['asset_id']))

1050

# Testing Sales

In [ ]:
# api endpoint
sales_endpoint = 'https://wax.api.atomicassets.io/atomicmarket'

In [ ]:
# add a state for sold, listed, etc.. ask the team about this 
# STATE parameter - NOT NEEDED
sales_key = 'https://wax.api.atomicassets.io/atomicmarket/v1/sales?asset_id=1099554263666'
url = sales_endpoint + sales_key
url

In [ ]:
x = requests.get('https://wax.api.atomicassets.io/atomicmarket/v1/sales?asset_id=1099554263666&state=3').json()['data']
x